In [1]:
import arviz as az
import pyro
import torch
import pyro.distributions as dist
from pyro.infer import MCMC, NUTS, Predictive
import pandas as pd
from torch.autograd import Variable
import numpy as np
import xarray
xarray.set_options(display_style="html");

pyro.enable_validation(True)
pyro.set_rng_seed(0)

In [2]:
#read data
data = pd.read_csv("linear_regression_data.csv", index_col=0)
time = data.time.values
slack_comments = torch.tensor(data.comments.values)
github_commits = torch.tensor(data.commits.values)
names = data.index.values
N = len(names)
data

,comments,commits,time
Alice,7500,25,4.5
Bob,10100,32,6.0
Cole,18600,49,7.0
Danielle,25200,66,12.0
Erika,27500,96,18.0


In [3]:
# data for out of sample predictions
candidate_devs = ["Francis", "Gerard"]
candidate_devs_time = torch.tensor([3.6, 5.1])
time = torch.tensor(time)
predictions_dict = {
    "developer": 2,
    "time_since_joined": candidate_devs_time
}

In [4]:
slack_comments = torch.tensor(data.comments.values)
github_commits = torch.tensor(data.commits.values)
time = torch.tensor(time)

dims={
    "slack_comments": ["developer"],
    "github_commits": ["developer"],
    "time": ["developer"],
}

data_dict = {
    "developer": N,
    "time_since_joined": time
}

def model(developer, time_since_joined):
    b_sigma = abs(pyro.sample('b_sigma', dist.Normal(0, 300)))
    c_sigma = abs(pyro.sample('c_sigma', dist.Normal(0, 6)))
    b0 = pyro.sample("b0", dist.Normal(0, 200))
    b1 = pyro.sample("b1", dist.Normal(0, 200))
    c0 = pyro.sample("c0", dist.Normal(0, 10))
    c1 = pyro.sample("c1", dist.Normal(0, 10))

    with pyro.plate('developer', developer):
        slack = pyro.sample("slack_comments", dist.Normal(b0 + b1 * time_since_joined, b_sigma), obs=slack_comments)
        github = pyro.sample("github_commits", dist.Normal(c0 + c1 * time_since_joined, c_sigma), obs=github_commits)
        return slack, github


nuts_kernel = NUTS(model, jit_compile=True, ignore_jit_warnings=True)
mcmc = MCMC(nuts_kernel, num_samples=400, warmup_steps=400,
            num_chains=4, disable_progbar=True)
mcmc.run(**data_dict)
posterior_samples = mcmc.get_samples()
posterior_predictive = Predictive(model, posterior_samples).forward(**data_dict)
prior = Predictive(model, num_samples=150).forward(**data_dict)

pyro_data = az.from_pyro(mcmc, prior=prior, posterior_predictive=posterior_predictive, coords={'developer': names}, dims=dims)

/home/nitish/env2/lib/python3.6/site-packages/ipykernel_launcher.py:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  This is separate from the ipykernel package so we can avoid doing imports until


In [5]:
pyro_data

Inference data with groups:
	> posterior
	> sample_stats
	> log_likelihood
	> posterior_predictive
	> observed_data
	> prior
	> prior_predictive

In [6]:
pyro_data.posterior

<xarray.Dataset>
Dimensions:  (chain: 4, draw: 400)
Coordinates:
  * chain    (chain) int64 0 1 2 3
  * draw     (draw) int64 0 1 2 3 4 5 6 7 8 ... 392 393 394 395 396 397 398 399
Data variables:
    b_sigma  (chain, draw) float32 -1633.9098 -1665.5798 ... -1517.8038
    c_sigma  (chain, draw) float32 5.7637467 5.1860986 ... -4.7391825 -5.993517
    b0       (chain, draw) float32 211.66737 333.29764 ... -96.43482 786.5245
    b1       (chain, draw) float32 1666.0728 1655.5056 ... 1613.7196 1581.1425
    c0       (chain, draw) float32 0.42486966 5.0319734 ... 2.768165 2.3824515
    c1       (chain, draw) float32 5.4298096 5.5872374 ... 5.5976653 5.3012886
Attributes:
    created_at:                 2020-02-17T14:58:31.900273
    inference_library:          pyro
    inference_library_version:  1.2.1

In [7]:
pyro_data.prior

<xarray.Dataset>
Dimensions:        (b0_dim_0: 1, b1_dim_0: 1, b_sigma_dim_0: 1, c0_dim_0: 1, c1_dim_0: 1, c_sigma_dim_0: 1, chain: 1, draw: 150)
Coordinates:
  * chain          (chain) int64 0
  * draw           (draw) int64 0 1 2 3 4 5 6 7 ... 143 144 145 146 147 148 149
  * b_sigma_dim_0  (b_sigma_dim_0) int64 0
  * c_sigma_dim_0  (c_sigma_dim_0) int64 0
  * b0_dim_0       (b0_dim_0) int64 0
  * b1_dim_0       (b1_dim_0) int64 0
  * c0_dim_0       (c0_dim_0) int64 0
  * c1_dim_0       (c1_dim_0) int64 0
Data variables:
    b_sigma        (chain, draw, b_sigma_dim_0) float32 -161.46501 ... -0.66203
    c_sigma        (chain, draw, c_sigma_dim_0) float32 3.5279593 ... 0.68555355
    b0             (chain, draw, b0_dim_0) float32 321.18915 ... -253.39551
    b1             (chain, draw, b1_dim_0) float32 85.571175 ... 169.98718
    c0             (chain, draw, c0_dim_0) float32 -6.7763753 ... 7.433421
    c1             (chain, draw, c1_dim_0) float32 10.421911 ... 6.5260434
Attributes:
    created_at:                 2020-02-17T14:58:31.987479
    inference_library:          pyro
    inference_library_version:  1.2.1

In [8]:
pyro_data.sample_stats

<xarray.Dataset>
Dimensions:    (chain: 4, draw: 400)
Coordinates:
  * chain      (chain) int64 0 1 2 3
  * draw       (draw) int64 0 1 2 3 4 5 6 7 ... 392 393 394 395 396 397 398 399
Data variables:
    diverging  (chain, draw) bool False False False False ... False False False
Attributes:
    created_at:                 2020-02-17T14:58:31.930724
    inference_library:          pyro
    inference_library_version:  1.2.1

In [9]:
pyro_data.log_likelihood

<xarray.Dataset>
Dimensions:         (chain: 4, developer: 5, draw: 400)
Coordinates:
  * chain           (chain) int64 0 1 2 3
  * draw            (draw) int64 0 1 2 3 4 5 6 7 ... 393 394 395 396 397 398 399
  * developer       (developer) object 'Alice' 'Bob' 'Cole' 'Danielle' 'Erika'
Data variables:
    slack_comments  (chain, draw, developer) float64 -8.326 -8.32 ... -8.906
    github_commits  (chain, draw, developer) float64 -2.671 -2.686 ... -2.755
Attributes:
    created_at:                 2020-02-17T14:58:31.952476
    inference_library:          pyro
    inference_library_version:  1.2.1

In [10]:
pyro_data.posterior_predictive

<xarray.Dataset>
Dimensions:         (chain: 4, developer: 5, draw: 400)
Coordinates:
  * chain           (chain) int64 0 1 2 3
  * draw            (draw) int64 0 1 2 3 4 5 6 7 ... 393 394 395 396 397 398 399
  * developer       (developer) object 'Alice' 'Bob' 'Cole' 'Danielle' 'Erika'
Data variables:
    slack_comments  (chain, draw, developer) int64 7500 10100 ... 25200 27500
    github_commits  (chain, draw, developer) int64 25 32 49 66 ... 32 49 66 96
Attributes:
    created_at:                 2020-02-17T14:58:31.969827
    inference_library:          pyro
    inference_library_version:  1.2.1

In [11]:
pyro_data.observed_data

<xarray.Dataset>
Dimensions:         (developer: 5)
Coordinates:
  * developer       (developer) object 'Alice' 'Bob' 'Cole' 'Danielle' 'Erika'
Data variables:
    slack_comments  (developer) int64 7500 10100 18600 25200 27500
    github_commits  (developer) int64 25 32 49 66 96
Attributes:
    created_at:                 2020-02-17T14:58:32.021253
    inference_library:          pyro
    inference_library_version:  1.2.1

In [12]:
pyro_data.prior_predictive

<xarray.Dataset>
Dimensions:         (chain: 1, developer: 5, draw: 150)
Coordinates:
  * chain           (chain) int64 0
  * draw            (draw) int64 0 1 2 3 4 5 6 7 ... 143 144 145 146 147 148 149
  * developer       (developer) object 'Alice' 'Bob' 'Cole' 'Danielle' 'Erika'
Data variables:
    slack_comments  (chain, draw, developer) int64 7500 10100 ... 25200 27500
    github_commits  (chain, draw, developer) int64 25 32 49 66 ... 32 49 66 96
Attributes:
    created_at:                 2020-02-17T14:58:32.004991
    inference_library:          pyro
    inference_library_version:  1.2.1

In [13]:
predictions = Predictive(model, posterior_samples).forward(**predictions_dict)
predictions

{'slack_comments': tensor([[ 7500, 10100, 18600, 25200, 27500],
         [ 7500, 10100, 18600, 25200, 27500],
         [ 7500, 10100, 18600, 25200, 27500],
         ...,
         [ 7500, 10100, 18600, 25200, 27500],
         [ 7500, 10100, 18600, 25200, 27500],
         [ 7500, 10100, 18600, 25200, 27500]]),
 'github_commits': tensor([[25, 32, 49, 66, 96],
         [25, 32, 49, 66, 96],
         [25, 32, 49, 66, 96],
         ...,
         [25, 32, 49, 66, 96],
         [25, 32, 49, 66, 96],
         [25, 32, 49, 66, 96]])}